<a href="https://colab.research.google.com/github/ashivashankars/Team_4Musketeers_capstone_project/blob/main/ScrapeFeatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Apply NLP techniques to extract 'Years of Experience', 'Diploma Type', 'Job Type', 'Sponsorship Requirement', and 'Skill Sets' from the text content in the `/content/all_internships_with_content.csv` file, and then save the augmented data to a new CSV file.

## Load Data

### Subtask:
Load the `/content/all_internships_with_content.csv` file into a pandas DataFrame and inspect its structure.


**Reasoning**:
Load the CSV file into a pandas DataFrame, display the first 5 rows, and print a concise summary to inspect its structure, as requested by the subtask instructions.



In [42]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
import pandas as pd
import csv
import sys

# Get the current field size limit
current_limit = csv.field_size_limit()

# Set a new, larger field size limit to handle large fields
csv.field_size_limit(sys.maxsize)

try:
    # Try reading the CSV with 'on_bad_lines='skip' to bypass problematic rows
    # Use header=None to prevent the first row from being interpreted as headers
    df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/all_internships_with_content.csv', engine='python', on_bad_lines='skip', header=None)

    # Set the first row as the header and then drop the first row
    df.columns = df.iloc[0]
    df = df[1:].reset_index(drop=True)

    print("First 20 rows of the DataFrame:")
    print(df.head())

    print("\nConcise summary of the DataFrame:")
    df.info()
finally:
    # Reset the field size limit to its original value
    csv.field_size_limit(current_limit)

First 20 rows of the DataFrame:
0                  Company                               Role  \
0  The Walt Disney Company                Data Product Intern   
1                Teamworks          Product Management Intern   
2                  Zscaler     Product Management Intern - AI   
3              🔥ServiceNow         Portfolio Associate Intern   
4                Santander  Digital Product Management Intern   

0         Location                                        Application Age  \
0  Celebration, FL  https://disney.wd5.myworkdayjobs.com/disneycar...  0d   
1    Remote in USA  https://ats.rippling.com/teamworks-careers/job...  0d   
2     San Jose, CA  https://job-boards.greenhouse.io/zscaler/jobs/...  1d   
3      Chicago, IL  https://jobs.smartrecruiters.com/ServiceNow/74...  1d   
4       Boston, MA  https://santander.wd3.myworkdayjobs.com/Santan...  1d   

0                                           raw_html  \
0  <!DOCTYPE html>\n<html lang="en-US">\n<head>\n...   
1 

## Extract Years of Experience (YOE)

### Subtask:
Develop and apply NLP techniques (e.g., regular expressions) to extract numeric values representing 'Years of Experience' from the text content. The extracted value will be converted to a numeric type.


### Subtask
Develop and apply NLP techniques (e.g., regular expressions) to extract numeric values representing 'Years of Experience' from the text content. The extracted value will be converted to a numeric type.

#### Instructions
1. Fill any missing values in the 'text_content' column with an empty string to ensure regex operations do not fail.
2. Define a regular expression pattern to capture numerical values related to 'years of experience'. Consider variations like 'X years', 'X+ years', 'X-Y years', or 'minimum X years'.
3. Apply the defined regex pattern to the 'text_content' column of the DataFrame `df` to extract potential years of experience.
4. From the extracted matches, determine the most appropriate single numeric value (e.g., the lower bound for ranges like '3-5 years', or the number itself for '3+ years').
5. Convert the extracted numeric values to an integer or float type.
6. Store the extracted and converted 'Years of Experience' in a new column named 'YOE' in the DataFrame `df`. If no experience is found, assign a default value like 0 or NaN.

**Reasoning**:
The first instruction is to fill any missing values in the 'text_content' column with an empty string to ensure that subsequent regex operations do not encounter errors. This is a crucial data preparation step before text processing.



In [44]:
df['text_content'] = df['text_content'].fillna('')
print("Missing values in 'text_content' after filling NaNs:")
print(df['text_content'].isnull().sum())

Missing values in 'text_content' after filling NaNs:
0


**Reasoning**:
Following the instructions, the next step is to define a regular expression pattern to capture various formats of 'years of experience' and apply it to the 'text_content' column. This will involve using `re.findall` or `str.findall` to get all potential matches for each entry.



In [45]:
import re

# Define a regex pattern to capture years of experience
# This pattern looks for numbers (X, X+, X-Y) followed by 'year(s)' with optional words in between.
# It also considers 'minimum X years' and similar phrases.
experience_pattern = re.compile(r'(\d+\+?|-?\d*\.?\d+\s*(?:to|-)\s*\d*\.?\d+|\d+)\s*(?:years?|yrs?|yr)\s*(?:of\s*experience|experience)?|minimum\s*(\d+)\s*(?:years?|yrs?|yr)')

# Apply the pattern to the 'text_content' column to find all matches
df['extracted_yoe_matches'] = df['text_content'].apply(lambda x: experience_pattern.findall(x.lower()))

print("First 5 rows of 'extracted_yoe_matches' column:")
print(df['extracted_yoe_matches'].head())

print("\nNumber of rows with extracted experience matches:")
print(df[df['extracted_yoe_matches'].apply(lambda x: len(x) > 0)].shape[0])

First 5 rows of 'extracted_yoe_matches' column:
0          []
1    [(3+, )]
2          []
3          []
4          []
Name: extracted_yoe_matches, dtype: object

Number of rows with extracted experience matches:
149


**Reasoning**:
The next step is to process the `extracted_yoe_matches` column to derive a single numeric value for 'Years of Experience' (YOE) for each entry, handling different formats like ranges or 'X+'. This involves iterating through the extracted matches, parsing them, and converting them to a numeric type, storing the result in the 'YOE' column.



In [46]:
def parse_yoe(matches):
    yoe_values = []
    for match_tuple in matches:
        # The regex has two main capturing groups: one for standard patterns and one for 'minimum X years'
        # We need to check both parts of the tuple
        for match_str in match_tuple:
            if match_str:
                # Handle ranges like '3-5 years' or '3 to 5 years'
                if 'to' in match_str or '-' in match_str:
                    parts = re.findall(r'\d+', match_str)
                    if parts: # Take the lower bound of the range
                        yoe_values.append(int(parts[0]))
                # Handle 'X+ years'
                elif '+' in match_str:
                    yoe_values.append(int(match_str.replace('+', '')))
                # Handle simple 'X years' or 'minimum X years'
                else:
                    try:
                        yoe_values.append(int(match_str))
                    except ValueError:
                        pass # Ignore non-numeric strings that might slip through

    if yoe_values:
        # Return the minimum years of experience found, as typically a requirement is 'at least X years'
        return min(yoe_values)
    return None # Return None if no valid YOE found

df['YOE'] = df['extracted_yoe_matches'].apply(parse_yoe)

# Fill any remaining None values with a default (e.g., 0 or NaN)
# Using NaN to distinguish between 'no experience found' and '0 years experience'
df['YOE'] = df['YOE'].fillna(0) # Default to 0 if nothing is found, as per instruction to assign 0 or NaN

print("First 5 rows of 'YOE' column:")
print(df['YOE'].head())

print("\nValue counts for 'YOE' column (showing non-zero values):")
print(df[df['YOE'] > 0]['YOE'].value_counts().sort_index())

First 5 rows of 'YOE' column:
0    0.0
1    3.0
2    0.0
3    0.0
4    0.0
Name: YOE, dtype: float64

Value counts for 'YOE' column (showing non-zero values):
YOE
1.0      35
2.0      20
3.0      12
5.0       1
6.0       4
8.0       1
10.0      3
15.0      1
16.0      1
18.0     25
25.0      1
28.0      2
30.0      2
35.0      1
40.0      6
50.0      1
65.0      1
70.0      1
130.0     1
175.0     1
Name: count, dtype: int64


## Extract Diploma Type

### Subtask:
Implement NLP logic to identify and categorize educational diplomas (e.g., 'None', 'Bachelor', 'Master', 'PhD') from the text. The extraction will map to the specified enum values.


#### Instructions Breakdown for Diploma Type Extraction

To effectively extract and categorize diploma types, I will follow these steps:

1.  **Define Diploma Types and Priority**: A dictionary will be created to store standardized diploma names (e.g., 'PhD', 'Master', 'Bachelor', 'Associate', 'High School', 'None') and their corresponding regular expression patterns. The order of keys in this dictionary will reflect the priority for extraction, from highest to lowest educational attainment.

2.  **Regular Expression Patterns**: For each diploma type, I will craft robust regular expression patterns that account for common abbreviations, full names, and variations (e.g., 'PhD', 'doctorate', 'master's degree', 'MS', 'bachelor', 'BS', 'high school diploma'). These patterns will be case-insensitive.

3.  **Iterative Application**: A function will be defined to iterate through the text content. It will attempt to match diploma patterns in the predefined priority order. The first match found will determine the 'Diploma Type' for that entry.

4.  **Assign to 'Diploma' Column**: The extracted diploma type will be stored in a new column named 'Diploma' in the DataFrame `df`.

5.  **Handle Missing Values**: If no diploma type is identified after checking all patterns, 'None' will be assigned to the 'Diploma' column to indicate no specific educational qualification was found or required.

6.  **Verify Extraction**: Finally, the value counts of the newly created 'Diploma' column will be printed to provide an overview of the distribution of extracted educational qualifications.

**Reasoning**:
Now, following the outlined plan, I will define the prioritized diploma types and their corresponding regular expression patterns in a dictionary. This will serve as the core logic for identifying educational qualifications within the text content.



In [47]:
import re

# Define diploma types and their regex patterns in order of priority (highest to lowest education)
diploma_patterns = {
    'PhD': [
        r'ph\.d\b',
        r'doctorate\b',
        r'doctoral\s*degree\b',
        r'doctor\s*of\s*philosophy\b'
    ],
    'Master': [
        r'master\.?\s*degree\b',
        r'm\.s\b',
        r'm\.a\b',
        r'mba\b',
        r'msc\b',
        r'graduate\s*degree\b'
    ],
    'Bachelor': [
        r'bachelor\.?\s*degree\b',
        r'b\.s\b',
        r'b\.a\b',
        r'bsc\b',
        r'undergraduate\s*degree\b'
    ],
    'Associate': [
        r'associate\.?\s*degree\b',
        r'a\.a\b',
        r'a\.s\b'
    ],
    'High School': [
        r'high\s*school\s*diploma\b',
        r'high\s*school\s*graduate\b',
        r'ged\b'
    ]
}

print("Diploma patterns defined successfully.")

Diploma patterns defined successfully.


**Reasoning**:
Following the defined `diploma_patterns`, I will now create a function to apply these patterns to the `text_content` column. This function will iterate through the patterns in order of priority, assign the first found diploma type to a new 'Diploma' column, and default to 'None' if no match is found, then print the value counts.



In [48]:
def extract_diploma(text):
    text = str(text).lower() # Ensure text is string and lowercased for case-insensitive matching
    for diploma_type, patterns in diploma_patterns.items():
        for pattern in patterns:
            if re.search(pattern, text):
                return diploma_type
    return 'None'

df['Diploma'] = df['text_content'].apply(extract_diploma)

print("Value counts for 'Diploma' column:")
print(df['Diploma'].value_counts())

Value counts for 'Diploma' column:
Diploma
None           1294
High School     198
Master           51
PhD              31
Bachelor          8
Name: count, dtype: int64


## Extract Job Type

### Subtask:
Use NLP to determine the job type (e.g., 'Intern', 'Full-Time', 'Both') from relevant text fields in the dataset. This will involve keyword matching and contextual analysis.


#### Reasoning:
To begin extracting 'Job Type', the first step is to define a comprehensive set of regular expression patterns for each target job type: 'Intern', 'Full-Time', 'Part-Time', and 'Contract'. These patterns will account for variations in wording and abbreviations. This setup is crucial for the subsequent NLP logic, allowing for accurate and prioritized identification of job types within the text content.

**Reasoning**:
Now, I will define the `job_type_patterns` dictionary with regex patterns for 'Intern', 'Full-Time', 'Part-Time', and 'Contract' job types, prioritizing 'Intern' and 'Full-Time' for more common scenarios as per the instructions.



In [49]:
import re

# Define job types and their regex patterns in order of priority
job_type_patterns = {
    'Intern': [
        r'internship\b',
        r'intern\b',
        r'co-op\b',
        r'coop\b'
    ],
    'Full-Time': [
        r'full-time\b',
        r'full\s*time\b',
        r'f/t\b'
    ],
    'Part-Time': [
        r'part-time\b',
        r'part\s*time\b',
        r'p/t\b'
    ],
    'Contract': [
        r'contract\b',
        r'freelance\b'
    ]
}

print("Job type patterns defined successfully.")

Job type patterns defined successfully.


**Reasoning**:
Now that the job type patterns are defined, I will create a function to iterate through the `text_content` column, apply these patterns in priority order, and assign the first matching job type to a new 'Job_type' column. If no match is found, it will default to 'None'.



In [50]:
def extract_job_type(text):
    text = str(text).lower() # Ensure text is string and lowercased for case-insensitive matching
    found_job_types = []

    # Check for 'Intern' first due to its common requirement for internships
    for pattern in job_type_patterns['Intern']:
        if re.search(pattern, text):
            found_job_types.append('Intern')
            # If 'Intern' is found, it's typically the primary job type for this dataset
            return 'Intern'

    # Check for other job types if 'Intern' is not found
    for job_type, patterns in job_type_patterns.items():
        if job_type == 'Intern': # Skip intern as it's already checked and returned if found
            continue
        for pattern in patterns:
            if re.search(pattern, text):
                found_job_types.append(job_type)

    if len(found_job_types) > 1:
        # If multiple non-intern job types are found, and no intern was found,
        # we need to decide what to return. For simplicity, we can prioritize or return 'Both'.
        # As the instruction is 'Both' or the most appropriate primary, let's return 'Both'
        # if more than one distinct type is detected (excluding 'Intern' as it's handled).
        # A more sophisticated logic might be needed here based on domain knowledge.
        # For this subtask, let's consider the first one found or 'Both' if ambiguous.
        if 'Full-Time' in found_job_types and 'Part-Time' in found_job_types:
            return 'Both'
        elif 'Full-Time' in found_job_types: # Prioritize Full-Time over others if present
            return 'Full-Time'
        elif 'Part-Time' in found_job_types:
            return 'Part-Time'
        elif 'Contract' in found_job_types:
            return 'Contract'
    elif len(found_job_types) == 1:
        return found_job_types[0]
    return 'None'

# Apply the function to the 'text_content' column
df['Job_type'] = df['text_content'].apply(extract_job_type)

print("Value counts for 'Job_type' column:")
print(df['Job_type'].value_counts())

Value counts for 'Job_type' column:
Job_type
None         905
Intern       549
Full-Time    106
Contract      21
Both           1
Name: count, dtype: int64


## Extract Sponsorship Requirement

### Subtask:
Apply NLP to identify if sponsorship is required (True/False or 1/0) based on keywords and phrases in the text content related to visa sponsorship.


#### Instructions
1. Define a list of regular expression patterns to identify phrases indicating a sponsorship requirement (e.g., 'visa sponsorship', 'sponsorship available', 'require sponsorship', 'work authorization'). These patterns should be case-insensitive.
2. Create a function that iterates through these patterns and checks if any match the 'text_content' for each row. The function should return `True` if any sponsorship-related phrase is found, and `False` otherwise.
3. Apply this function to the 'text_content' column of the DataFrame `df`.
4. Store the boolean result in a new column named 'Require_Sponsorship' in the DataFrame `df`.
5. Print the value counts of the 'Require_Sponsorship' column to show the distribution of sponsorship requirements.

**Reasoning**:
Following the instructions, the first step is to define a list of regular expression patterns to identify various phrases indicating sponsorship requirements. This forms the basis for the NLP extraction.



In [51]:
import re

# Define regex patterns for sponsorship keywords
sponsorship_patterns = [
    r'visa\s*sponsorship',
    r'sponsorship\s*available',
    r'require\s*sponsorship',
    r'work\s*authorization',
    r'eligible\s*to\s*work\s*in\s*the\s*us\s*without\s*sponsorship',
    r'only\s*us\s*citizens\s*or\s*green\s*card\s*holders',
    r'us\s*citizen',
    r'green\s*card\s*holder',
    r'no\s*sponsorship',
    r'not\s*sponsor',
    r'can\s*not\s*sponsor'
]

print("Sponsorship patterns defined successfully.")

Sponsorship patterns defined successfully.


**Reasoning**:
Now that the sponsorship patterns are defined, I will create a function to check if any of these patterns are present in the 'text_content' for each row, apply this function to create the 'Require_Sponsorship' column, and then print its value counts as per the instructions.



In [52]:
def check_sponsorship(text):
    text = str(text).lower()  # Ensure text is string and lowercased

    # First, check for phrases indicating NO sponsorship (higher priority if present)
    no_sponsorship_patterns = [
        r'no\s*sponsorship',
        r'not\s*sponsor',
        r'can\s*not\s*sponsor',
        r'eligible\s*to\s*work\s*in\s*the\s*us\s*without\s*sponsorship'
    ]
    for pattern in no_sponsorship_patterns:
        if re.search(pattern, text):
            return False # Explicitly states no sponsorship

    # Then, check for phrases indicating sponsorship might be required or is available
    # or implying a need for specific work authorization that may lead to sponsorship issues
    sponsorship_needed_patterns = [
        r'visa\s*sponsorship',
        r'sponsorship\s*available',
        r'require\s*sponsorship',
        r'work\s*authorization',
        r'only\s*us\s*citizens\s*or\s*green\s*card\s*holders',
        r'us\s*citizen',
        r'green\s*card\s*holder'
    ]
    for pattern in sponsorship_needed_patterns:
        if re.search(pattern, text):
            return True # Indicates sponsorship is relevant

    return False # Default if no strong indicators are found

df['Require_Sponsorship'] = df['text_content'].apply(check_sponsorship)

print("Value counts for 'Require_Sponsorship' column:")
print(df['Require_Sponsorship'].value_counts())

Value counts for 'Require_Sponsorship' column:
Require_Sponsorship
False    1321
True      261
Name: count, dtype: int64


# Task
Consolidate and refine the skill pattern dictionary, incorporating all existing and new skill categories and their regex patterns into a single, comprehensive definition. Then, re-extract the skill sets from the 'raw_html' column of the DataFrame using this consolidated dictionary, summarize the extracted skills by listing the top 20 most frequent and the total unique skills, and finally, save the updated DataFrame to `augmented_internships_data.csv`.

## Consolidate Skill Patterns

### Subtask:
Create a single code cell that defines the complete `skill_patterns` dictionary. This dictionary will group specific skills under broader categories (e.g., 'Software Engineering (Backend & DevOps)', 'Business & Data Analysis') where they logically fit, and keep general technical skills (like Python, Java, R, Git) and soft skills as their own top-level entries or in new categories like 'Linux/Unix', 'Project Management & Agile', 'Communication Skills', 'Problem Solving', and 'Critical Thinking'. All regex patterns will be integrated into this single, comprehensive dictionary, eliminating redundancy from previous definitions.


**Reasoning**:
I will define a single, comprehensive `skill_patterns` dictionary in a new code cell, incorporating all previously defined skills, grouped categories, and new categories as specified in the instructions. This consolidates the skill pattern definitions and ensures all regex patterns are included.



In [53]:
import re

# Define the comprehensive skill_patterns dictionary
skill_patterns = {

    # ====================================================================
    # 💻 CORE PROGRAMMING & DATABASES
    # ====================================================================

    'Python': [r'python(?!.*(django|flask|data|ml))', r'pythons', r'\bscripting\s*languages', r'\bscripting\b'],
    'Java/JVM': [r'java(?!script)', r'jvm', r'spring\s*boot', r'hibernate', r'kotlin'],
    'JavaScript/TypeScript': [r'javascript', r'js', r'typescript', r'ts\b', r'nodejs', r'node\.js', r'npm', r'yarn', r'strong\s*programming\s*skills\s*in\s*typescript/javascript'],
    'C/C++': [r'c\+\+', r'cpp\b', r'c\s*plus\s*plus', r'\bc\b(?!#)', r'experience\s*with\s*c\+\+'],
    'Go/Rust': [r'go\s*lang', r'golang', r'rust'],
    'C#/.NET': [r'c\#', r'c\s*sharp', r'\.net\b', r'dot\s*net'],

    'SQL/Databases': [r'sql', r'mysql', r'postgresql', r'sqlite', r'database\s*query', r'data\s*warehousing', r'relational\s*database', r'understanding\s*of\s*sql'],
    'NoSQL/NewSQL': [r'nosql', r'no-sql', r'mongodb', r'cassandra', r'redis', r'dynamodb', r'understanding\s*of\s*no-sql\s*databases'],

    # ====================================================================
    # 💻 SOFTWARE ENGINEERING (FULL STACK, DEVOPS, QA)
    # ====================================================================

    # Frontend & Full Stack
    'Software Engineering (Frontend & Full Stack)': [
        r'frontend', r'full\s*stack\s*perspectives', r'full\s*stack', r'ui/ux', r'user\s*interface',
        r'react', r'react\.js', r'reactjs', r'nextjs', r'gatsby', r'angular', r'angular\.js', r'vue(\.js)?',
        r'client-side', r'web\s*applications', r'web\s*technologies\s*and\s*concepts'
    ],
    'HTML/CSS': [r'html\d*', r'css\d*', r'sass', r'less', r'scss', r'webpack', r'babel', r'strong\s*programming\s*skills\s*in\s*.*css'],

    # Web Technologies & Architecture
    'API/Architecture': [r'api(s)?', r'rest\s*api', r'graphql', r'microservices', r'architecture', r'system\s*design', r'backend\s*components', r'implementation\s*of\s*scalable\s*and\s*secure\s*software\s*solutions'],
    'Web Technologies & Protocols': [r'web\s*technologies\s*and\s*concepts', r'http/s', r'jwt', r'rest\s*apis', r'graphql', r'design\s*systems'],
    'Mobile Development': [r'ios\s*development', r'android\s*development', r'swift', r'kotlin', r'objective\s*c', r'flutter', r'react\s*native', r'mobile\s*application', r'xcode'],

    # DevOps, CI/CD, & Observability
    'DevOps & CI/CD': [r'devops', r'ci/cd', r'continuous\s*integration', r'continuous\s*deployment', r'jenkins', r'gitlab\s*ci', r'github\s*actions', r'pipeline(s)?', r'build\s*automated\s*ci/cd\s*pipelines'],
    'Infrastructure as Code (IaC)': [r'terraform', r'ansible', r'chef', r'puppet', r'iac\b', r'cloudformation'],
    'SRE/Observability': [r'site\s*reliability\s*engineer', r'sre\b', r'monitoring', r'logging', r'observability', r'code\s*observability', r'prometheus', r'grafana', r'splunk', r'elk\s*stack', r'root\s*cause\s*analysis'],
    'Containerization/Orchestration': [r'docker', r'kubernetes', r'k8s', r'container(s|ization)', r'helm\s*chart', r'ecs', r'openshift'],
    'Cloud Platforms (AWS/Azure/GCP)': [r'aws\b', r'amazon\s*web\s*services', r'azure', r'microsoft\s*azure', r'gcp\b', r'google\s*cloud\s*platform', r'cloud\s*computing', r'lambda', r'ec2', r's3'],

    # Testing & Quality Assurance
    'Automated Testing & QA': [
        r'automated\s*testing', r'qa\b', r'quality\s*assurance', r'application\s*testing\s*solutions', r'testing\b', r'debugging\b',
        r'unit\s*test(s)?', r'integration\s*test(s)?', r'e2e\s*testing', r'selenium', r'jest', r'cypress', r'code\s*reviews',
        r'software\s*automation\s*framework', r'build\s*tools\s*for\s*automated\s*testing', r'validate\s*system\s*safety', r'detect\s*performance\s*regressions'
    ],

    # ====================================================================
    # 📱 PRODUCT MANAGEMENT
    # ====================================================================
    'Product Strategy/Roadmap': [r'product\s*management', r'product\s*manager', r'pm\b', r'product\s*vision', r'roadmap', r'product\s*strategy', r'go-to-market', r'market\s*analysis', r'prioritized\s*backlog\s*items', r'digital\s*portfolio'],
    'User/Customer Focus': [r'user\s*stories', r'prds', r'user\s*research', r'customer\s*journey', r'voice\s*of\s*the\s*customer', r'persona(s)?', r'usability'],
    'Metrics & Analysis (PM)': [r'kpis', r'metrics', r'ab\s*testing', r'data-driven\s*decisions', r'experimentation', r'funnel\s*analysis', r'retention', r'engagement', r'data\s*mining\s*and\s*analysis'],

    # ====================================================================
    # 🤖 DATA SCIENCE, AI & MACHINE LEARNING
    # ====================================================================
    'ML/DL Concepts': [r'machine\s*learning', r'ml\b', r'deep\s*learning', r'dl\b', r'artificial\s*intelligence', r'ai\b', r'predictive\s*modeling', r'model\s*validation', r'model\s*deployment', r'mle\b', r'basic\s*understanding\s*of\s*machine\s*learning\s*concepts\s*and\s*algorithms'],
    'Statistical Modeling': [r'statistics', r'statistical\s*analysis', r'hypothesis\s*testing', r'regression', r'bayesian'],
    'Specialized AI Fields': [r'natural\s*language\s*processing', r'nlp\b', r'computer\s*vision', r'cv\b', r'reinforcement\s*learning', r'generative\s*ai', r'large\s*language\s*model', r'llm\b'],
    'Data Science Tools': [r'tensorflow', r'pytorch', r'scikit-learn', r'keras', r'hugging\s*face', r'pandas', r'numpy', r'scipy', r'jupyter', r'rstudio'],
    'Big Data & Processing': [r'spark', r'hadoop', r'hive', r'presto', r'kafka', r'etl\b', r'data\s*pipeline', r'data\s*lake', r'databricks'],
    'Data Analysis/Viz': [r'data\s*science', r'data\s*analysis', r'data\s*analytics', r'tableau', r'power\s*bi', r'data\s*visualization', r'looker', r'deriving\s*insights\s*from\s*large\s*datasets'],

    # ====================================================================
    # 📈 QUANTITATIVE FINANCE & FINTECH
    # ====================================================================
    'Quantitative Finance Core': [r'quantitative\s*finance', r'quant\s*analyst', r'financial\s*modeling', r'algorithmic\s*trading', r'high\s*frequency\s*trading', r'hft\b'],
    'Financial Instruments': [r'derivatives', r'options', r'futures', r'fixed\s*income', r'equities', r'fx\b', r'commodities', r'securities'],
    'Math/Statistics (Quant)': [r'time\s*series\s*analysis', r'stochastic\s*calculus', r'monte\s*carlo\s*simulation', r'portfolio\s*optimization', r'black-scholes', r'calibration'],
    'FinTech Domain': [r'domain\s*knowledge\s*of\s*the\s*futures\s*trading\s*industry', r'futures\s*trading', r'trading\s*systems', r'exchange\s*protocols', r'fintech'],
    'Risk/Compliance': [r'risk\s*management', r'market\s*risk', r'credit\s*risk', r'regulatory\s*compliance', r'basel\s*\d', r'risk\s*culture'],

    # ====================================================================
    # 🔧 HARDWARE & SYSTEMS ENGINEERING
    # ====================================================================
    'Hardware & Systems Engineering': [r'aerospace\s*engineering', r'electrical\s*engineering', r'ee\b', r'system\s*safety', r'delivery\s*system\s*features', r'hardware\s*engineering', r'circuit\s*design', r'pcb\s*design'],
    'Semiconductors & IC': [r'fpga', r'asic', r'semiconductor', r'vlsi', r'ic\s*design', r'verilog', r'vhdl', r'cadence', r'synopsys'],
    'Systems/Firmware': [r'embedded\s*systems', r'firmware', r'microcontroller', r'rtos', r'real-time', r'signal\s*processing', r'dsp\b'],
    'Simulation & Analysis': [r'simulation(s)?', r'system\s*level\s*simulations', r'simulation\s*tools', r'analyzing\s*real\s*world\s*data', r'incorporating\s*additional\s*signals', r'analysis\s*tools', r'execute\s*millions\s*of\s*simulation\s*scenarios'],

    # ====================================================================
    # 🤝 UNIVERSAL SKILLS & METHODOLOGIES
    # ====================================================================
    'Core Engineering Concepts': [r'algorithms', r'data\s*structures', r'object-oriented\s*programming', r'oop\b', r'functional\s*programming', r'design\s*patterns', r'participate\s*in\s*code\s*reviews'],
    'Project Management & Agile': [r'agile', r'scrum', r'kanban', r'project\s*management', r'complexity/time\s*estimation', r'project\s*tracking', r'sprint', r'backlog\s*grooming'],
    'Business/Analysis': [r'business\s*analyst', r'data\s*analysis', r'business\s*intelligence', r'bi\b', r'requirements\s*gathering', r'process\s*improvement', r'jira', r'confluence', r'business\s*needs', r'business\s*requirements'],
    'Communication & Soft Skills': [r'communication\s*skills', r'written\s*communication', r'verbal\s*communication', r'teamwork\s*skills', r'collaboration', r'problem-solving\s*skills', r'critical\s*thinking', r'attention\s*to\s*detail', r'work\s*independently', r'manage\s*time\s*effectively'],
    'Version Control/CLI': [r'git', r'github', r'gitlab', r'bitbucket', r'version\s*control', r'linux', r'unix', r'shell\s*scripting', r'bash', r'command\s*line'],
    'Content Management': [r'contentful', r'cms\b', r'content\s*management\s*system'],
}

print("Consolidated skill patterns defined successfully with new additions.")

Consolidated skill patterns defined successfully with new additions.


**Reasoning**:
Now that the comprehensive `skill_patterns` dictionary has been defined, I need to re-apply the `extract_skills` function to the DataFrame's 'raw_html' column to correctly populate the 'Skill_Sets' column with the newly consolidated and expanded skill definitions. I will then print the head of the 'Skill_Sets' column and the count of rows with extracted skills to verify the update.



In [54]:
def extract_skills(text):
    text = str(text).lower()  # Convert to lowercase for case-insensitive matching
    found_skills = set()  # Use a set to store unique skills

    for skill_name, patterns in skill_patterns.items():
        for pattern in patterns:
            if re.search(pattern, text):
                found_skills.add(skill_name)
                # Break from inner loop once a pattern for this skill is found
                break
    return sorted(list(found_skills)) # Return a sorted list of unique skills

# Apply the function to the 'raw_html' column as specified in the original task (4314a9c4 used raw_html)
df['Skill_Sets'] = df['raw_html'].apply(extract_skills)

print("First 5 rows of re-extracted 'Skill_Sets' column with consolidated patterns:")
print(df['Skill_Sets'].head())

print("\nNumber of rows with extracted skill sets (after re-extraction with consolidated patterns):")
print(df[df['Skill_Sets'].apply(len) > 0].shape[0])

First 5 rows of re-extracted 'Skill_Sets' column with consolidated patterns:
0    [Big Data & Processing, Business/Analysis, Clo...
1    [API/Architecture, Automated Testing & QA, Big...
2    [API/Architecture, Business/Analysis, C#/.NET,...
3    [API/Architecture, Automated Testing & QA, Bus...
4    [Business/Analysis, Communication & Soft Skill...
Name: Skill_Sets, dtype: object

Number of rows with extracted skill sets (after re-extraction with consolidated patterns):
1524


**Reasoning**:
Now that the 'Skill_Sets' column has been re-extracted with the consolidated patterns, I will re-aggregate all extracted skills, count their frequencies, print the top 20 most frequent skills, and display the total number of unique skills found across the entire dataset to reflect these changes.



In [55]:
from collections import Counter

# Flatten the list of lists in 'Skill_Sets' to get all individual skills
all_skills_updated = [skill for sublist in df['Skill_Sets'] for skill in sublist]

# Count the frequency of each skill
skill_counts_updated = Counter(all_skills_updated)

print("\nTop 20 most frequently extracted skills (after update):")
for skill, count in skill_counts_updated.most_common(20):
    print(f"- {skill}: {count}")

print(f"\nTotal number of unique skills found across the dataset (after update): {len(skill_counts_updated)}")


Top 20 most frequently extracted skills (after update):
- HTML/CSS: 1524
- JavaScript/TypeScript: 1524
- ML/DL Concepts: 1524
- Hardware & Systems Engineering: 1379
- API/Architecture: 1187
- Cloud Platforms (AWS/Azure/GCP): 1004
- C/C++: 934
- Version Control/CLI: 880
- Communication & Soft Skills: 834
- User/Customer Focus: 832
- Automated Testing & QA: 735
- Specialized AI Fields: 716
- Software Engineering (Frontend & Full Stack): 655
- Go/Rust: 640
- Financial Instruments: 623
- C#/.NET: 583
- Business/Analysis: 575
- Semiconductors & IC: 528
- Data Analysis/Viz: 474
- Core Engineering Concepts: 397

Total number of unique skills found across the dataset (after update): 43


## Save Augmented Data (Final)

### Subtask:
Save the DataFrame, now with skill sets extracted using the consolidated patterns, to the `augmented_internships_data.csv` file, ensuring all changes are permanently stored.


**Reasoning**:
I will save the DataFrame `df` to a CSV file named `augmented_internships_data.csv`, excluding the index, to permanently store all extracted and augmented data.



In [56]:
output_file_path = 'augmented_internships_data.csv'
df.to_csv(output_file_path, index=False)
print(f"Augmented DataFrame successfully saved to: {output_file_path}")

Augmented DataFrame successfully saved to: augmented_internships_data.csv


In [57]:
selected_columns = [
    'Company',
    'Role',
    'Location',
    'Application',
    'YOE',
    'Diploma',
    'Job_type',
    'Require_Sponsorship',
    'Skill_Sets'
]

filtered_df = df[selected_columns].copy()

output_file_path_filtered = 'filtered_internships_data.csv'
filtered_df.to_csv(output_file_path_filtered, index=False)

print(f"Filtered DataFrame successfully saved to: {output_file_path_filtered}")
print("First 5 rows of the filtered DataFrame:")
print(filtered_df.head())

Filtered DataFrame successfully saved to: filtered_internships_data.csv
First 5 rows of the filtered DataFrame:
0                  Company                               Role  \
0  The Walt Disney Company                Data Product Intern   
1                Teamworks          Product Management Intern   
2                  Zscaler     Product Management Intern - AI   
3              🔥ServiceNow         Portfolio Associate Intern   
4                Santander  Digital Product Management Intern   

0         Location                                        Application  YOE  \
0  Celebration, FL  https://disney.wd5.myworkdayjobs.com/disneycar...  0.0   
1    Remote in USA  https://ats.rippling.com/teamworks-careers/job...  3.0   
2     San Jose, CA  https://job-boards.greenhouse.io/zscaler/jobs/...  0.0   
3      Chicago, IL  https://jobs.smartrecruiters.com/ServiceNow/74...  0.0   
4       Boston, MA  https://santander.wd3.myworkdayjobs.com/Santan...  0.0   

0      Diploma Job_type  Req